In [161]:
import pandas as pd
import numpy as np
import pymysql
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from jsonpath import jsonpath
import json

In [162]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [163]:
sql = '''
select om.order_number,
tprm.all_rental 总租金, tod.new_actual_money 签约价,tod.new_actual_money-tprm.all_rental 买断尾款,toe.expressage
from db_digua_business.t_order om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_express toe on toe.order_id=om.id
where  om.status=4 and om.order_type='JD_ORDER'
'''
df = query(sql)

In [164]:
sql_req = '''
select distinct o.order_number, od.actual_money 买断价, date(tojr.create_time) 同步日期 from db_digua_business.t_order as o 
left join db_digua_business.t_order_details as od on o.id = od.order_id 
left join db_digua_business.t_order_jd_request tojr on tojr.order_number=o.order_number
where tojr.create_time>=date_add(current_date, interval -7 day)
and tojr.request_json like '%IN_THE_LEASE%' 
'''
df_req = query(sql_req)
df_req = df_req.sort_values(by='同步日期', ascending=False).groupby('order_number').head(1)
# df_req.loc[:, '同步日期'] = df_req.create_time.dt.strftime('%Y-%m-%d')
# df_req.drop_duplicates(subset=['order_number'], inplace=True)

In [176]:
# # 计算gmv
# df_req.loc[:, '同步日期'] = pd.to_datetime(df_req.同步日期)
# df_req_new = df_req[(df_req.同步日期>=pd.to_datetime('2025-06-12'))&(df_req.同步日期<=pd.to_datetime('2025-06-20'))]
# df_req_new.groupby('同步日期').agg({'买断价': 'sum'}).to_excel('F:/需求/总需求/京东gmv.xlsx')


In [160]:
df = df[~df.expressage.isna()]
# df.loc[:, '签收时间'] = df.expressage.apply(lambda x: jsonpath(json.loads(x), '$.routes'))
# 自定义函数来解析expressage列并筛选opCode为80或100的acceptTime
def extract_accept_times(expressage):
    try:
        data = json.loads(expressage)
        for route in data['routes']:
            if route['opCode'] in ['80', '100']:
                return route['acceptTime']
    except:
        return np.nan
df.loc[:, '签收时间'] = df.expressage.apply(extract_accept_times)
df.loc[:, '签收日期'] = pd.to_datetime(df.签收时间).dt.strftime('%Y-%m-%d')
df.loc[:, '签收月份'] = pd.to_datetime(df.签收时间).dt.strftime('%Y-%m')
# df_new = df[df.签收时间=='2025-06']
# df_new = df[(df.签收日期>='2025-07-01')&(df.签收日期<='2025-07-06')]
df_new = df_req.merge(df, on='order_number', how='left')
df_new_g = df_new.groupby('同步日期').agg({'order_number':'count', '总租金': 'sum', '买断价': 'sum', '买断尾款': 'sum'}).rename(columns={'order_number': '数量'})
df_new_g.loc['合计', :] = df_new_g.sum(axis=0)
today = datetime.now().strftime('%Y%m%d')
with pd.ExcelWriter(f'F:/需求/瑞彤需求/签收时间订单_{today}.xlsx', engine='xlsxwriter') as writer:
    df_new_g.to_excel(writer, sheet_name='签收时间数据')